In [27]:
%%bash
pip install stringcase
pip install tqdm

You are using pip version 8.1.1, however version 8.1.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 8.1.1, however version 8.1.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [8]:
import sys

sys.path.insert(0, '..')
import rx

import re
import copy
import os

import stringcase
from tqdm import tqdm
from jinja2 import Template


operator_index_template = Template("""
.. !!!AUTO!!! BEGIN 'Operator Index'

{% for operator in operators -%}
* |{{operator}}|
{% endfor %}

.. toctree::
   :hidden:
    
   {% for group in groups -%}
   {{group}}
   {% endfor %}
    
.. !!!AUTO!!! END 'Operator Index'""")


operator_file_template = Template(""".. !!!AUTO!!! (remove this comment to edit)

.. figure:: /img/RxPY/misc/under-construction-icon.png
    :align: center
    
    Under construction...

.. currentmodule:: rx

{% for operator in operators -%}
.. _operator_{{operator.name}}:
{% endfor %}

{{title}}

{% for operator in operators %}
.. automethod:: Observable.{{operator.name}}

    {% for pic in operator.pics %}
    .. image:: /img/reactivex/operators/{{pic}}
        :align: center
    {% endfor %}
{% endfor %}
""")

operator_index_re = re.compile(
    "\n.. !!!AUTO!!! BEGIN 'Operator Index'\n(.+)\n.. !!!AUTO!!! END 'Operator Index'",
    re.DOTALL) # Match newlines!

def update_operator_index(operators, groups):
    with open('api/operators/index.rst', 'r') as f:
        contents = f.read()

    groupnames = sorted(list(groups.keys()))
    new_operator_index = operator_index_template.render(operators=operators,
                                                        groups=groupnames)
    
    if re.search(operator_index_re, contents) is None:
        raise Exception("Badly formatted index delimiters.")
    
    updated_contents = re.sub(operator_index_re, new_operator_index, contents)
    
    with open('api/operators/index.rst', 'w') as f:
        f.write(updated_contents)


def best_matches(operator, basenames):
    best = []
    camelcase = stringcase.camelcase(operator)
    capcamel = stringcase.camelcase(camelcase)
    
    good_match = False
    
    for match in basenames:
        if match == operator:
            best.append(match)
            good_match = True
    
    for match in basenames:
        if match == operator:
            pass
        elif match == camelcase and good_match:
            pass
        else:
            best.append(match)
    
    return best

    
def matches_operator(operator: str, base: str, other_operators: set):
    
    camelcase = stringcase.camelcase(operator)
    capcamel = stringcase.camelcase(camelcase)
    
    for opname in [operator, camelcase, capcamel]:
        if base == opname:
            return 0
        elif re.match(r"\w\.{}\.".format(opname), base):
            return 1
        elif re.match(r"{}\d+".format(opname), base):
            return 2

        if base.startswith(opname):
            other_matches = [matches_operator(op, base, set())
                               for op in other_operators]

            if all(match is None for match in other_matches):
                return 3
    
    else:
        return None
    
def find_pics(operator, files, dirpath, other_operators):
    pics = [[], [], [], []]
    for basename in files:
        base, ext = os.path.splitext(basename)
        if os.path.isfile(os.path.join(dirpath, basename)):
            match = matches_operator(operator, base, other_operators)
            if match is not None:
                pics[match].append(base)
                                
    unfiltered = [basename for sublist in pics for basename in sublist]
    filtered = best_matches(operator, unfiltered)
    return [base + '.png' for base in filtered]
        
def write_operator_file(basename, operators, all_operators, force=False):
    title = basename.replace('_', ' ').capitalize()
    underline = "=" * len(title)
    underlined_title = title + '\n' + underline
    filename = os.path.join('api', 'operators', basename + '.rst')
    try:
        with open(filename, 'r') as fin:
            contents = fin.read()
    except FileNotFoundError:
        contents = ""
    
    pics_dirpath = os.path.join('img', 'reactivex', 'operators')
    pics_basenames = sorted(os.listdir(pics_dirpath))
    
    operators_ = [dict(name=op,
                       pics=find_pics(op,
                                      pics_basenames,
                                      pics_dirpath,
                                      all_operators - set(op)))
                    for op in operators]
    
    if (not contents) or contents.startswith('.. !!!AUTO!!!') or force:
        with open(filename, 'w') as fout:
            fout.write(operator_file_template.render(operators=operators_,
                                                     title=underlined_title))

def write_operator_files(groups, all_operators, force=False):
    for groupname, operators in tqdm(sorted(groups.items())):
        write_operator_file(groupname, operators, all_operators, force=force)
    
def complete_groups(operators, groups, op_to_group):
    all_groups = copy.deepcopy(groups)
    for operator in operators:
        if operator not in op_to_group:
            all_groups[operator] = [operator]
            
    return all_groups
    
def make_op_to_group(groups):
    op_to_group = {}
    for key, value in groups.items():
        for val in value:
            op_to_group[val] = key
    return op_to_group

def write_operator_aliases_file(operators):
    with open('api/operators/operator-aliases.rst', 'w') as f:
        f.write(":orphan:\n\n")
        for operator in operators:
            f.write(".. |{0}| replace:: :ref:`{0} <operator_{0}>`\n" \
                        .format(operator))

        
groups = {
    'all': ['all', 'every'],
    'and-then-when': ['and_', 'then_do', 'when'],
    'buffer': ['buffer', 'buffer_with_count', 'buffer_with_time',
               'buffer_with_time_or_count', 'pairwise'],
    'catch': ['catch_exception', 'on_error_resume_next'],
    'combine_latest': ['combine_latest', 'with_latest_from'],
    'concat': ['concat_all', 'concat'],
    'contains': ['contains', 'find', 'find_index', 'is_empty', 'some'],
    'create': ['create', 'generate', 'generate_with_relative_time'],
    'debounce': ['debounce', 'throttle_with_selector', 'throttle_with_timeout'],
    'distinct' : ['distinct', 'distinct_until_changed'],
    'delay': ['delay', 'delay_subscription', 'delay_with_selector'],
    'case': ['case', 'defer', 'if_then', 'switch_case'],
    'do': ['do_action', 'finally_action', 'tap'],
    'first': ['first', 'first_or_default', 'single', 'single_or_default'],
    'flat_map': ['expand', 'flat_map', 'for_in', 'many_select', 'select_many', 'select_switch'], # concat_all
    'filter': ['filter', 'slice', 'where'],
    'empty-never-throw': ['empty', 'never', 'throw', 'throw_exception'],
    'element_at': ['element_at', 'element_at_or_default'],
    'from': ['from_', 'from_callback', 'from_future',
             'from_iterable', 'from_list', 'of'],
    'group_by': ['group_by', 'group_by_until', 'partition'],
    'join': ['join', 'group_join'],
    'just': ['just', 'return_value'],
    'last': ['last', 'last_or_default'],
    'materialize-dematerialize': ['dematerialize', 'materialize'],
    'max': ['max', 'max_by'],
    'merge': ['merge', 'merge_all', 'merge_observable'],
    'publish': ['let', 'let_bind', 'multicast', 'publish', 'publish_value'],
    'repeat': ['repeat', 'repeat_infinitely'],
    'reduce': ['aggregate', 'expand', 'reduce'],
    'retry': ['retry', 'retry_infinitely'],
    'sample': ['sample', 'throttle_first', 'throttle_last'],
    'skip': ['skip', 'skip_with_time'],
    'skip_last': ['skip_last', 'skip_last_with_time'],
    'skip_until': ['skip_until' 'skip_until_with_time'],
    'skip_while': ['skip_while', 'skip_while_with_index'],
    'start': ['start', 'start_async', 'to_async'],
    'take': ['take', 'take_with_time'],
    'switch': ['exclusive', 'select_switch', 'switch_latest'],
    'take_last': ['take_last', 'take_last_buffer'],
    'take_while': ['take_while', 'take_while_with_index'],
    'empty-never-throw': ['empty', 'never', 'throw', 'throw_exception'],
    'timeout': ['timeout', 'timeout_with_selector'],
    'to': ['to_async', 'to_blocking', 'to_dict', 'to_future',
           'to_iterable', 'to_list', 'to_set'],
    'window': ['window', 'window_with_count', 'window_with_time', 'window_with_time_or_count'],
    'zip': ['zip', 'zip_array', 'zip_list'],
}

operators = [name for name in dir(rx.Observable) if not name.startswith('_')]

op_to_group = make_op_to_group(groups)
all_groups = complete_groups(operators, groups, op_to_group)
write_operator_files(all_groups, set(operators), force=False)
update_operator_index(operators, all_groups)
write_operator_aliases_file(operators)

100%|██████████| 85/85 [00:42<00:00,  1.67it/s]


In [5]:
'skip_until' in dir(rx.Observable)

True